In [ ]:
# Imports
import numpy as np
from scipy.integrate import solve_ivp
from scipy import sparse
import matplotlib.pyplot as plt
from matplotlib import animation

In [ ]:
# Define initial conditions of detector
l = 2 # length of wire, meters
mass = 1 # total mass of wire
tensile_str = 1 # Tensile Strength of wire
dia = 1 # diameter of wire
cs_area = np.pi * dia**2 / 4 # cross sectional area of wire

mu = mass / l # # linear mass density
tension = tensile_str * cs_area # Tension on wire
young_mod = 1 # elastic modulus of wire
v_press = np.sqrt(young_mod / mu) # wavespeed for longitudinal wave
v_shear = np.sqrt(tension / mu) # wavespeed for transverse waves

# Impact conditions of particle and track
G = 1 # gravitational constant
M = 1 # mass of inciting particle
v_p = 1 # velocity of inciting particle

dz = .001 # mm step size for wire?
z0 = 1 # position of particle track and wire 'intersection', meters
b = .05 # shortest distance between wire and particle track, meters
theta = 0 # Angle between particle track and z (wire) axes, radians
phi = 0 # Angle between particle track and x axes, radians
zwire = np.arange(0 - z0, l - z0, dz) # define wire along z axes s.t. z0 is at z_string == 0
v_0 = 2 * G * M / (v_p * (b**2 + (zwire * np.sin(theta))**2)) # shared component of initial velocities

# Set up time erray for ivp solution
dt = .000001 # mus step size for time?
t0 = 0.0
tf = 0.01    # 10 ms total time
t_eval = np.arange(t0, tf, dt)


In [ ]:
# Define needed functions
arr_dim = zwire.size
laplacian = sparse.diags([1, -2, 1], [-1, 0, 1], shape= (arr_dim, arr_dim)) / dz**2

def couple_fun(t, u):
    """" 
    Function that returns first derivative in time => 
        u is coupled position and velocity =>
            First arr_dim points return velocity: second N points of u
            Second arr_dim points return accelleration: v_p^2 * Laplacian.[first N points of u]
    """
    u[0] = 0 # fixed ends
    u[arr_dim - 1] = 0 #fixed ends
    du = np.zeros(2 * arr_dim) 
    # velocity is second half of coupled array (u)
    du[:arr_dim] = u[arr_dim:] 
    du[0] = 0 # fixed ends
    du[arr_dim - 1] = 0 # fixed ends
    # acceleration is wave eq on first half of coupled array (u)
    du[arr_dim:] = v_p**2 * laplacian.dot(u[:arr_dim]) 
    du[arr_dim] = 0 # fixed ends
    du[-1] = 0 # fixed end
    return du

def accell_mix(t, sol):
    """
    Take solve_ivp coupled solution and a given time and return acceleration using 
        finite difference favouring symmetric difference (first and last frame use
        increasing and decreasing difference respectively)
    """
    d = 2
    if t == 0:
        less_dt = sol.y[arr_dim:, t]
        d = 1
    else:
        less_dt = sol.y[arr_dim:, t - 1]
    if t == len(sol.t) - 1:
        more_dt = sol.y[arr_dim:, t]
        d = 1
    else:
        more_dt = sol.y[arr_dim:, t + 1]  
    return (more_dt - less_dt) / (d * dt)

In [ ]:
# z axes wave 
v_z = - v_0 * zwire * np.sin(theta)**2 

In [ ]:
# x axes wave 
v_x = v_0 * zwire * np.sin(theta) * np.cos(theta)

In [ ]:
# y axes wave
v_y = v_0 * b